In [23]:
import os
import sys
import glob
import json
import pandas as pd
import numpy as np
import string
from snowballstemmer import TurkishStemmer
import pprint
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix




# READ JSON

# from snowballstemmer import TurkishStemmer

# df = [
#     'müşteri',
#     'hizmetlerine',
#     'cabuk',
#     'baglaniyorum',
#     'konuda',
#     'yardımcı',
#     'oluyorlar',
#     'islemlerimde'
# ]
# turkStem = TurkishStemmer()
# data_words_nostops = [turkStem.stemWord(word) for word in df]
# print(data_words_nostops)

def has_key(dict, key):
    if key in dict.keys():
        return 1
    return 0

labels = {}
def readLabels(fileName):
    f = open(file, "r", encoding='utf-8')
    jsonData = json.loads(f.read())

    if (not has_key(labels, jsonData["Suç"])):
        labels[jsonData["Suç"]] = 1    
    else :
        labels[jsonData["Suç"]] = labels[jsonData["Suç"]] + 1




path = "C://Users/onkol/Desktop/2021-01-20220322T055600Z-001/2021-01/*.json"
files = glob.glob(path)


for file in files:
    readLabels(file)
    
sort_orders = sorted(labels.items(), key=lambda x: x[1], reverse=True)
# jsonString = json.dumps(sort_orders, indent = 4, ensure_ascii=False)
# jsonFile = open("data.json", "w", encoding='utf-8')
# jsonFile.write(jsonString)
# jsonFile.close()

classes = list(map(lambda x: x[0], sort_orders))
#print(classes[1:])
dict_contents = {}
for file in files:
    f = open(file, "r", encoding='utf-8')
    jsonData = json.loads(f.read())

    key = jsonData["Suç"] if jsonData["Suç"] != '' else "others"
    if (has_key(dict_contents, key)):
        dict_contents[key].append(jsonData["ictihat"])
    else :
        dict_contents[key] = [jsonData['ictihat']]
        
        
kasten_yaralama = dict_contents[' Kasten yaralama']
uyusturucu_madde_ticareti = dict_contents[' Uyuşturucu madde ticareti yapma']
hirsizlik =  dict_contents[' Hırsızlık']
tehdit =  dict_contents[' Tehdit']
sayili_kanuna_muhalefet =  dict_contents[' 5607 sayılı Kanuna muhalefet']
hakaret =  dict_contents[' Hakaret']
uyusturucu_madde_bulundurma =  dict_contents[' Kullanmak için uyuşturucu madde bulundurma']
taksirle_yaralama = dict_contents[' Taksirle yaralama']
silahli_teror_orgutu_uyeligi = dict_contents[' Silahlı terör örgütüne üye olma']
yasa_6136_aykiriligi =  dict_contents[' 6136 sayılı Yasaya aykırılık']
others =  dict_contents['others']


df = pd.DataFrame(columns = ["kasten_yaralama", "uyusturucu_madde_ticareti", "hirsizlik", "tehdit", "sayili_kanuna_muhalefet", 
                "hakaret", "uyusturucu_madde_bulundurma", "taksirle_yaralama", "silahli_teror_orgutu_uyeligi", "yasa_6136_aykiriligi", "others"])


df['kasten_yaralama'] = pd.Series(kasten_yaralama)
df['uyusturucu_madde_ticareti'] = pd.Series(uyusturucu_madde_ticareti)
df['hirsizlik'] = pd.Series(hirsizlik)
df['tehdit'] = pd.Series(tehdit)
df['sayili_kanuna_muhalefet'] = pd.Series(sayili_kanuna_muhalefet)
df['hakaret'] = pd.Series(hakaret)
df['uyusturucu_madde_bulundurma'] = pd.Series(uyusturucu_madde_bulundurma)
df['taksirle_yaralama'] = pd.Series(taksirle_yaralama)
df['silahli_teror_orgutu_uyeligi'] = pd.Series(silahli_teror_orgutu_uyeligi)
df['yasa_6136_aykiriligi'] = pd.Series(yasa_6136_aykiriligi)
df['others'] = pd.Series(others)

#df

df.to_csv("train_set.csv", encoding='utf-8', index=False)
select_cols = ["kasten_yaralama","uyusturucu_madde_ticareti","hirsizlik","tehdit","sayili_kanuna_muhalefet","hakaret","uyusturucu_madde_bulundurma","taksirle_yaralama","silahli_teror_orgutu_uyeligi","yasa_6136_aykiriligi","others"]
x = df[select_cols]
#x = x.transpose()
#print(x.shape)
y = np.array([0,1,2,3,4,5,6,7,8,9,10])
#print(y.shape)
from sklearn.model_selection import train_test_split  #Importing the train and test function
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=36)

from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(x_train, y_train)
y_pred = svclassifier.predict(x_test)

from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


In [24]:
path = "C://Users/onkol/Desktop/2021-01-20220322T055600Z-001/2021-01/*.json"
files = glob.glob(path)
ictihats = []
suc_array = []

for file in files:
    f = open(file, "r", encoding='utf-8')
    jsonData = json.loads(f.read())
    ictihats.append(jsonData['ictihat'].strip()[:20])
    suc = jsonData["Suç"] if jsonData["Suç"] != '' else "others"
    suc_array.append(suc.strip())

ictihats_lower = []
for i in ictihats:
    i = i.lower()
    ictihats_lower.append(i)
    
ictihats_punctuation = []

for i in ictihats_lower:
    i = i.translate(str.maketrans("", "", string.punctuation))
    ictihats_punctuation.append(i)

x_train, x_test, y_train, y_test = train_test_split(ictihats_punctuation, 
                                                    suc_array, 
                                                    test_size=0.20,random_state=36)

#print(len(suc_array))
#print(ictihats_punctuation)
# index_list = []
# for i in range(0,100):
#     index_list.append(str(i))

# preprocessed_ictihats = []
# for i in ictihats_punctuation:
#        preprocessed_ictihats.append(list(filter(lambda x: x != "",i.split(" "))))

# #print(preprocessed_ictihats)
# turkStem = TurkishStemmer()
# for index, ictihat in enumerate(preprocessed_ictihats,start=0):
#     preprocessed_ictihats[index] = [turkStem.stemWord(word) for word in ictihat]
    
#print(preprocessed_ictihats)
file = open("stopwords.txt", "r", newline='', encoding='utf-8')
result = file.read()
stopwords = word_tokenize(result)

tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= stopwords)
training_data = tfidfvectorizer.fit_transform(x_train)
testing_data = tfidfvectorizer.transform(x_test)

#tfidf_wm_array
#tfidf_wm_array.shape
# frequency_matrix = pd.DataFrame(tfidf_wm_array, columns=tfidfvectorizer.get_feature_names())
# from sklearn.model_selection import train_test_split
# y = np.array([0,1,2,3,4,5,6,7,8,9,10]) 


naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

predictions = naive_bayes.predict(testing_data)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions,
                                           average='macro')))
print('Recall score: ', format(recall_score(y_test, predictions
                                           ,average='macro')))
print('F1 score: ', format(f1_score(y_test, predictions
                                           ,average='macro')))

print(confusion_matrix(y_test,predictions))


# from sklearn.svm import SVC
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(x_train, y_train)
# y_pred = svclassifier.predict(x_test)

# from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))



# frequency_matrix
# print(tfidfvectorizer.vocabulary_)
# print(tfidf_wm.toarray())
# tfidf_tokens = tfidfvectorizer.get_feature_names()
# df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = index_list,columns = tfidf_tokens)
# print("\nTD-IDF Vectorizer\n")
# print(df_tfidfvect)
# vectorizer.get_feature_names_out()
# ictihat_freq = []


# for i in preprocessed_ictihats:
#     count_freq = Counter(i)
#     ictihat_freq.append(count_freq)
    
# pprint.pprint(ictihat_freq)



C:\Users\onkol\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lahî'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Accuracy score:  0.5857117214144678


C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision score:  0.012132333291898412
Recall score:  0.011382016116547789
F1 score:  0.009712902473178735
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
